In [1]:
#import packeges
import numpy as np
from matplotlib import image
from matplotlib import pyplot as plt
from keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split
import keras
from random import randint
from random import sample
from random import shuffle
import pickle
import albumentations as A

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Flatten
from keras.callbacks import ModelCheckpoint
import keras
from tensorflow.python.keras.utils.data_utils import Sequence

In [3]:
#load prepared data
with open('train test split data.pkl', 'rb') as f: 
    X_train, X_val, y_train, y_val =  pickle.load(f)

In [4]:
classes = np.arange(125)

In [5]:
#augmentation

augment = A.Compose([

    A.HorizontalFlip(always_apply = False, p = 0.5),

    A.VerticalFlip(always_apply = False, p = 0.1),

    A.RGBShift(r_shift_limit = np.random.uniform(0.01, 0.1), 
            g_shift_limit = np.random.uniform(0.01, 0.1), 
            b_shift_limit = np.random.uniform(0.01, 0.1),
            always_apply = False, p = 0.5),

    A.RandomBrightnessContrast(brightness_limit = np.random.uniform(0.1, 0.2), 
                                contrast_limit = np.random.uniform(0.1, 0.5), 
                                brightness_by_max = [True, False][randint(0,1)],
                                always_apply = False, p = 0.2),

    A.ColorJitter(brightness = np.random.uniform(0, 0.6), 
                  contrast = np.random.uniform(0, 0.6), 
                  saturation = np.random.uniform(0, 0.6), 
                  hue = np.random.uniform(0, 0.6), 
                  always_apply = False, p = 0.5),

    A.RandomGamma(gamma_limit = (80, 200), always_apply = False, p = 0.2),

    A.ToGray(always_apply = False, p = 0.2),

    A.ToSepia(always_apply = False, p = 0.2),

    A.Transpose(always_apply = False, p = 0.3),

    A.ElasticTransform(alpha = randint(1,4), sigma = randint(1,4), 
                        alpha_affine = randint(1,4), 
                        interpolation = randint(0, 4), 
                        border_mode = randint(0, 4), 
                        value = np.random.uniform(0, 1), 
                        mask_value = np.random.uniform(0, 1),
                        always_apply = False, p = 0.2),

    A.ShiftScaleRotate(shift_limit = np.random.uniform(0, 0.15), 
                        scale_limit = np.random.uniform(0, 0.15), 
                        rotate_limit = 45, interpolation = randint(0, 4), 
                        border_mode = randint(0, 4), 
                        value = np.random.uniform(0, 1), 
                        mask_value = np.random.uniform(0, 1), 
                        shift_limit_x = None, shift_limit_y = None,
                        always_apply = False, p = 0.2),
                     
    A.Resize(224, 224, interpolation = 2, always_apply=True),

    A.ToFloat(max_value=255, always_apply=True)

])

resize = A.Compose([
          A.Resize(224, 224, interpolation = 2, always_apply=True),
          A.ToFloat(max_value=255, always_apply=True)
])


In [6]:
#batch generator

class batch_generator(Sequence):

    def __init__(self, x_set, y_set, batch_size, augmentations, shuffle):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.augment = augmentations
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        if(self.shuffle):
            nums = sample(range(len(self.x)), self.batch_size)
            batch_x = [self.x[num_i] for num_i in nums]
            batch_y = [self.y[num_i] for num_i in nums]
        else:
            batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.stack([
            self.augment(image = np.uint8(x * 255))["image"] for x in batch_x
        ], axis=0), np.array(batch_y)

In [7]:
# load VGG without last layers
from keras.applications import VGG19
pretrained = VGG19(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')

In [8]:
for layer in pretrained.layers:
    layer.trainable = False

In [9]:
model = Sequential()

model.add(pretrained)

model.add(Flatten())

model.add(Dense(125, activation="relu", kernel_regularizer = keras.regularizers.l1_l2(l1 = 0.01)))

model.add(Dense(125, activation='softmax'))

model.build()

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 125)               3136125   
_________________________________________________________________
dense_1 (Dense)              (None, 125)               15750     
Total params: 23,176,259
Trainable params: 3,151,875
Non-trainable params: 20,024,384
_________________________________________________________________


In [11]:
checkpoint = ModelCheckpoint("chk.hdf5", monitor='val_categorical_accuracy', verbose=1,
    save_best_only=True, mode='auto')

In [12]:
model.compile(loss = "categorical_crossentropy", optimizer = keras.optimizers.Nadam(learning_rate=0.001), 
              metrics=["categorical_accuracy"])

In [13]:
#history = model.fit(train_gen, epochs = 50, 
#                              validation_data = val_gen, 
#                              validation_steps = 203, callbacks=[checkpoint1, checkpoint2]) 